<a href="https://colab.research.google.com/github/20520566-uit/IE212-LAB02-20520566/blob/main/IE212_LAB02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install pyspark

FILE UPDATE


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving users.txt to users.txt


SPARK SESSION

In [44]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("DS200_Lab2").setMaster("local[*]")
sc = SparkContext(conf=conf)

movies_path = "movies.txt"
ratings1_path = "ratings_1.txt"
ratings2_path = "ratings_2.txt"
users_path = "users.txt"
occupation_path = "occupation.txt"

In [45]:
movies_raw = sc.textFile(movies_path)
ratings1_raw = sc.textFile(ratings1_path)
ratings2_raw = sc.textFile(ratings2_path)
users_raw = sc.textFile(users_path)
occupation_raw = sc.textFile(occupation_path)

# Hàm tách an toàn (loại bỏ header/nếu có dòng rỗng)
def split_line(line):
    return [x.strip() for x in line.split("::")] if "::" in line else [x.strip() for x in line.split(",")]

In [46]:
# --- Movies: MovieID,Title,Genres  ---
def parse_movie(line):
    if not line: return None
    # tách theo :: trước, nếu không thì comma
    if "::" in line:
        parts = line.split("::")
    else:
        parts = line.split(",")
    if len(parts) < 3:
        return None
    movieid = parts[0].strip()
    title = parts[1].strip()
    genres = parts[2].strip()
    return (movieid, (title, genres))

movies = movies_raw.map(parse_movie).filter(lambda x: x is not None)
# movie lookup dict RDD -> (movieid -> title)
movieTitles = movies.mapValues(lambda x: x[0])  # (movieid, title)
movieGenres = movies.mapValues(lambda x: x[1])  # (movieid, genres)

# --- Ratings: UserID,MovieID,Rating,Timestamp ---
def parse_rating(line):
    if not line: return None
    parts = line.strip().split()
    # try common separators
    if len(parts) == 1:
        #comma or ::
        if "::" in line:
            parts = line.split("::")
        else:
            parts = line.split(",")
    # normalize to 4 fields
    if len(parts) < 3:
        return None
    # Nhận formats: UserID,MovieID,Rating[,Timestamp] or with ::
    # Lấy ít nhất 3 fields -> uid, mid, rating (timestamp)
    uid = parts[0].strip()
    mid = parts[1].strip()
    rating = float(parts[2].strip())
    ts = parts[3].strip() if len(parts) >= 4 else None
    return (uid, mid, rating, ts)

ratings = ratings1_raw.union(ratings2_raw).map(parse_rating).filter(lambda x: x is not None)
# RDD of (uid, mid, rating, ts)

In [47]:
movie_ratings = ratings.map(lambda x: (x[1], (x[2], 1)))  # (movieID, (rating,1))
movie_sum_count = movie_ratings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
# (movieID, (sumRatings, count))

# average
movie_avg = movie_sum_count.mapValues(lambda sc: (sc[0]/sc[1], sc[1]))  # (movieID, (avg, count))

# join title
movie_with_title = movie_avg.join(movieTitles)  # (movieID, ((avg,count), title))
# Map to (title, avg, count)
movie_final = movie_with_title.map(lambda x: (x[1][1], round(x[1][0][0], 4), x[1][0][1]))

# Phim có ít nhất 5 votes
qualified = movie_final.filter(lambda x: x[2] >= 5)

# Sort by average rating (descending), title (ascending)
sorted_qualified = qualified.sortBy(lambda x: (-x[1], x[0]))

# Tất cả các phim có ít nhất 5 votes
if sorted_qualified.count() > 0:
    for title, avg, cnt in sorted_qualified.collect():
        print(f"  {title} - AverageRating: {avg} (TotalRatings: {cnt})")

    # Phim đc rate cao nhất
    top_movie = sorted_qualified.first()
    title, avg, cnt = top_movie
    print(f"\n{title} is the highest rated movie with an average rating of {avg} among movies with at least 5 ratings.")
else:
    print("Không có phim nào thỏa điều kiện >=5 ratings.")

  Sunset Boulevard (1950) - AverageRating: 4.3571 (TotalRatings: 7)
  The Terminator (1984) - AverageRating: 4.0556 (TotalRatings: 18)
  The Godfather: Part II (1974) - AverageRating: 4.0 (TotalRatings: 17)
  No Country for Old Men (2007) - AverageRating: 3.8889 (TotalRatings: 18)
  The Lord of the Rings: The Fellowship of the Ring (2001) - AverageRating: 3.8889 (TotalRatings: 18)
  The Social Network (2010) - AverageRating: 3.8571 (TotalRatings: 7)
  The Lord of the Rings: The Return of the King (2003) - AverageRating: 3.8182 (TotalRatings: 11)
  E.T. the Extra-Terrestrial (1982) - AverageRating: 3.6667 (TotalRatings: 18)
  Gladiator (2000) - AverageRating: 3.6111 (TotalRatings: 18)
  Fight Club (1999) - AverageRating: 3.5 (TotalRatings: 7)
  Mad Max: Fury Road (2015) - AverageRating: 3.4722 (TotalRatings: 18)
  Lawrence of Arabia (1962) - AverageRating: 3.4444 (TotalRatings: 18)
  The Silence of the Lambs (1991) - AverageRating: 3.1429 (TotalRatings: 7)

Sunset Boulevard (1950) is th

In [48]:
# Bài 2
# movieGenres : (movieID, genres_string)
movie_genres_expanded = movieGenres.flatMap(lambda x: [(x[0], g) for g in x[1].split("|") if g])  # (movieID, genre)
# join ratings by movieID: ratings map (mid, rating)
ratings_mid_rating = ratings.map(lambda x: (x[1], x[2]))  # (movieID, rating)

# join to get (movieID, (rating, genre)) then map to (genre, (rating,1))
genre_ratings = ratings_mid_rating.join(movie_genres_expanded) \
    .map(lambda x: (x[1][1], (x[1][0], 1)))  # (genre, (rating,1))

genre_sum_count = genre_ratings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
genre_avg = genre_sum_count.mapValues(lambda sc: (round(sc[0]/sc[1], 4), sc[1]))

# Output
for genre, (avg, cnt) in genre_avg.collect():
    print(f"{genre} - AverageRating: {avg} (TotalRatings{cnt})")

Drama - AverageRating: 3.7578 (TotalRatings128)
Action - AverageRating: 3.713 (TotalRatings54)
Sci-Fi - AverageRating: 3.7315 (TotalRatings54)
Biography - AverageRating: 3.56 (TotalRatings25)
Family - AverageRating: 3.6667 (TotalRatings18)
Horror - AverageRating: 4.0 (TotalRatings2)
Fantasy - AverageRating: 3.8621 (TotalRatings29)
Thriller - AverageRating: 3.7037 (TotalRatings27)
Mystery - AverageRating: 4.0 (TotalRatings2)
Adventure - AverageRating: 3.6325 (TotalRatings83)
Film-Noir - AverageRating: 4.3571 (TotalRatings7)
Crime - AverageRating: 3.8095 (TotalRatings42)


In [49]:
# Bài 3
# Parse users: UserID,Gender,Age,Occupation,Zip-code
def parse_user(line):
    if not line: return None
    parts = line.strip().split(",") if "," in line else line.split("::")
    if len(parts) < 2: return None
    uid = parts[0].strip()
    gender = parts[1].strip()
    age = parts[2].strip() if len(parts) > 2 else None
    occupation = parts[3].strip() if len(parts) > 3 else None
    return (uid, (gender, age, occupation))

users = users_raw.map(parse_user).filter(lambda x: x is not None)  # (uid,(gender,age,occ))

# ratings: (uid, mid, rating, ts) -> map to (uid, (mid, rating))
ratings_by_user = ratings.map(lambda x: (x[0], (x[1], x[2])))

# join
joined = ratings_by_user.join(users)  # (uid, ((mid,rating), (gender,age,occ)))
# map to ((mid, gender), (rating,1))
mid_gender_rating = joined.map(lambda x: ((x[1][0][0], x[1][1][0]), (x[1][0][1], 1)))

sumcount_mid_gender = mid_gender_rating.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
# Map to (mid, (gender, avg, count))
mid_gender_avg = sumcount_mid_gender.map(lambda x: (x[0][0], (x[0][1], round(x[1][0]/x[1][1],4), x[1][1])))
# Gộp sử dụng mid cho cả 2 GT
from collections import defaultdict
mid_grouped = mid_gender_avg.groupByKey().mapValues(lambda vals: {v[0]: (v[1], v[2]) for v in vals})
# Join with titles
mid_with_title_gender = mid_grouped.join(movieTitles)  # (mid, ( {gender: (avg,count), ...}, title))
# Output
def format_gender_avgs(mapping):
    male = mapping.get('M') or mapping.get('Male') or (None,0)
    female = mapping.get('F') or mapping.get('Female') or (None,0)
    return male, female

results = mid_with_title_gender.map(lambda x: (x[1][1],  # title
                                               format_gender_avgs(x[1][0])
                                              ))
# Print
for title, (male, female) in results.collect():
    m_avg, m_cnt = male if male else (None,0)
    f_avg, f_cnt = female if female else (None,0)
    print(f"{title} - Male_Avg: {m_avg}, Female_Avg: {f_avg}")


Gladiator (2000) - Male_Avg: 3.5909, Female_Avg: 3.6429
Sunset Boulevard (1950) - Male_Avg: 4.3333, Female_Avg: 4.5
Mad Max: Fury Road (2015) - Male_Avg: 4.0, Female_Avg: 3.3214
The Social Network (2010) - Male_Avg: 4.0, Female_Avg: 3.6667
E.T. the Extra-Terrestrial (1982) - Male_Avg: 3.8125, Female_Avg: 3.55
Fight Club (1999) - Male_Avg: 3.5, Female_Avg: 3.5
The Terminator (1984) - Male_Avg: 3.9286, Female_Avg: 4.1364
The Lord of the Rings: The Fellowship of the Ring (2001) - Male_Avg: 4.0, Female_Avg: 3.8
The Godfather: Part II (1974) - Male_Avg: 4.0556, Female_Avg: 3.9375
The Lord of the Rings: The Return of the King (2003) - Male_Avg: 3.75, Female_Avg: 3.9
Psycho (1960) - Male_Avg: None, Female_Avg: 4.0
The Silence of the Lambs (1991) - Male_Avg: 3.3333, Female_Avg: 3.0
No Country for Old Men (2007) - Male_Avg: 3.9167, Female_Avg: 3.8333
Lawrence of Arabia (1962) - Male_Avg: 3.55, Female_Avg: 3.3125


In [50]:
# Bài 4
def age_group(age):
    try:
        a = int(age)
    except:
        return "unknown"
    if a <= 18:
        return "0-18"
    elif a <= 35:
        return "18-35"
    elif a <= 50:
        return "35-50"
    else:
        return "50+"

# users: (uid,(gender, age, occ))
users_agegroup = users.mapValues(lambda v: age_group(v[1]))

# ratings_by_user is (uid,(mid,rating))
ratings_user_join_age = ratings_by_user.join(users_agegroup)  # (uid, ((mid,rating), agegroup))
# map to ((mid, agegroup), (rating,1))
mid_age_rating = ratings_user_join_age.map(lambda x: ((x[1][0][0], x[1][1]), (x[1][0][1], 1)))
mid_age_sumcount = mid_age_rating.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
# Map to (mid, (agegroup, avg))
mid_age_avg = mid_age_sumcount.map(lambda x: (x[0][0], (x[0][1], round(x[1][0]/x[1][1],4))))
# group by mid
mid_age_grouped = mid_age_avg.groupByKey().mapValues(dict)  # (mid, {agegroup: avg, ...})
# join titles
mid_age_with_title = mid_age_grouped.join(movieTitles)  # (mid, ( {agegroup:avg}, title))

# Print
for mid, (groupmap, title) in mid_age_with_title.collect():
    out = {
        "0-18": groupmap.get("0-18"),
        "18-35": groupmap.get("18-35"),
        "35-50": groupmap.get("35-50"),
        "50+": groupmap.get("50+")
    }
    print(f"{title} - [0-18: {out['0-18']}, 18-35: {out['18-35']}, 35-50: {out['35-50']}, 50+: {out['50+']}]")


Gladiator (2000) - [0-18: None, 18-35: 3.4375, 35-50: 3.8125, 50+: 3.5]
Mad Max: Fury Road (2015) - [0-18: None, 18-35: 3.3636, 35-50: 3.6429, 50+: None]
Sunset Boulevard (1950) - [0-18: None, 18-35: 4.1667, 35-50: 4.5, 50+: None]
The Social Network (2010) - [0-18: None, 18-35: 4.0, 35-50: 3.6667, 50+: None]
E.T. the Extra-Terrestrial (1982) - [0-18: None, 18-35: 3.5625, 35-50: 3.8333, 50+: 3.0]
Fight Club (1999) - [0-18: None, 18-35: 3.5, 35-50: 3.5, 50+: 3.5]
The Terminator (1984) - [0-18: None, 18-35: 4.1667, 35-50: 4.05, 50+: 3.75]
The Lord of the Rings: The Fellowship of the Ring (2001) - [0-18: None, 18-35: 4.0, 35-50: 3.8333, 50+: None]
The Godfather: Part II (1974) - [0-18: None, 18-35: 3.7778, 35-50: 4.25, 50+: None]
The Lord of the Rings: The Return of the King (2003) - [0-18: None, 18-35: 3.8333, 35-50: 3.8125, 50+: None]
Psycho (1960) - [0-18: None, 18-35: 4.5, 35-50: 3.5, 50+: None]
The Silence of the Lambs (1991) - [0-18: None, 18-35: 3.0, 35-50: 3.25, 50+: None]
No Count

In [52]:
# Bài 5
# occupation_raw assumed: "ID,Occupation" or "ID::Occupation"
def parse_occupation(line):
    if "::" in line:
        parts = line.split("::")
    else:
        parts = line.split(",")
    if len(parts) < 2: return None
    return (parts[0].strip(), parts[1].strip())

occupations = occupation_raw.map(parse_occupation).filter(lambda x: x is not None)  # (id, occupation_name)
occ_map = occupations.collectAsMap()  # small file -> collect OK

# users currently: (uid, (gender, age, occupation_field))
# occupation_field might be id or name; check and convert
def user_occ_name(user_tuple):
    gender, age, occ = user_tuple
    occ_name = occ
    if occ in occ_map:
        occ_name = occ_map[occ]
    return (gender, age, occ_name)

users_with_occname = users.mapValues(user_occ_name)

# join ratings_by_user -> (uid, ((mid,rating), (gender,age,occ_name)))
ratings_user_occ = ratings_by_user.join(users_with_occname)
# map to (occupation, (rating,1))
occ_ratings = ratings_user_occ.map(lambda x: (x[1][1][2], (x[1][0][1], 1)))
occ_sumcount = occ_ratings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
occ_avg = occ_sumcount.mapValues(lambda sc: (sc[1], round(sc[0]/sc[1],4)))  # (occ, (totalRatings, avg))
for occ, (total, avg) in occ_avg.collect():
    print(f"{occ} - AverageRating: {avg}, (TotalRatings: {total})")


Designer - AverageRating: 4.0, (TotalRatings: 13)
Student - AverageRating: 4.0, (TotalRatings: 8)
Journalist - AverageRating: 3.8529, (TotalRatings: 17)
Accountant - AverageRating: 3.5833, (TotalRatings: 6)
Artist - AverageRating: 3.7273, (TotalRatings: 11)
Lawyer - AverageRating: 3.6471, (TotalRatings: 17)
Nurse - AverageRating: 3.8636, (TotalRatings: 11)
Programmer - AverageRating: 4.25, (TotalRatings: 10)
Salesperson - AverageRating: 3.6471, (TotalRatings: 17)
Doctor - AverageRating: 3.6905, (TotalRatings: 21)
Consultant - AverageRating: 3.8571, (TotalRatings: 14)
Engineer - AverageRating: 3.5556, (TotalRatings: 18)
Manager - AverageRating: 3.4688, (TotalRatings: 16)
Teacher - AverageRating: 3.7, (TotalRatings: 5)


In [53]:
# Bài 6
# ratings: (uid, mid, rating, ts)
def ts_to_year(ts):
    if ts is None: return None
    try:
        # nếu timestamp là integer unix (seconds)
        t = int(ts)
        year = datetime.datetime.fromtimestamp(t).year
        return str(year)
    except:
        # parse dạng "YYYY-MM-DD..." hoặc lấy 4 ký tự đầu
        try:
            return ts.strip()[:4]
        except:
            return None

ratings_with_year = ratings.map(lambda x: (ts_to_year(x[3]), x[2])).filter(lambda x: x[0] is not None)
# (year, rating)
year_sumcount = ratings_with_year.map(lambda x: (x[0], (x[1], 1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
year_stats = year_sumcount.mapValues(lambda sc: (sc[1], round(sc[0]/sc[1],4)))  # (year, (total, avg))
for year, (total, avg) in sorted(year_stats.collect(), key=lambda x: x[0]):
    print(f"{year} - TotalRatings: {total}, AverageRating: {avg}")


1577 - TotalRatings: 4, AverageRating: 3.875
1578 - TotalRatings: 24, AverageRating: 3.75
1579 - TotalRatings: 23, AverageRating: 3.7826
1580 - TotalRatings: 23, AverageRating: 3.7826
1581 - TotalRatings: 24, AverageRating: 3.75
1582 - TotalRatings: 22, AverageRating: 3.6818
1583 - TotalRatings: 24, AverageRating: 3.75
1584 - TotalRatings: 22, AverageRating: 3.7727
1585 - TotalRatings: 13, AverageRating: 3.7308
1586 - TotalRatings: 5, AverageRating: 3.7


SPARK SESSION END

In [43]:
if 'sc' in globals() and sc:
    sc.stop()
    print("SparkContext stopped.")
else:
    print("No active SparkContext to stop.")

SparkContext stopped.
